## 스튜디오 촬영 데이터 기준 데이터 전처리

## 0. F만 남기고 다 삭제

In [1]:
import os
import shutil

In [6]:
root = 'C:/Users/omocomo/Downloads/' # 아래 데이터 폴더가 있는 경로
DATA = 'SENTENCE_REAL001' 

#### 영상 F 빼고 지우기

In [6]:
path = root + DATA + '/영상/'
files = os.listdir(path)
for file in files:
    file_name, ext = os.path.splitext(file)
    if file_name[-1]=='F': # F가 아닌 파일
        continue
    os.remove(path+file)

#### 형태소Json F 빼고 지우기

In [7]:
path = root + DATA + '/형태소Json/'
files = os.listdir(path)
for file in files:
    file_name, ext = os.path.splitext(file)
    file_name.split('_')
    if file_name.split('_')[4]=='F': # F가 아닌 파일
        continue
    os.remove(path+file)

#### 키포인트Json F 빼고 지우기(폴더 삭제)

In [8]:
path = root + DATA + '/키포인트Json/'
folders = os.listdir(path)
for folder in folders:
    folder_name, ext = os.path.splitext(folder)
    if folder_name[-1]=='F': # F가 아닌 폴더
        continue
    shutil.rmtree(path+folder)

## 1. 영상에서 이미지 Frame 추출

In [8]:
import json
import math

#### 형태소 Json에서 start, end 지점 찾기!

In [9]:
video = [] # 영상 명
start = [] # 시작 지점
end = [] # 끝 지점
path = root + DATA + '/형태소Json/'
files = os.listdir(path)
for file in files:
    with open(path+file, encoding='UTF-8') as json_file:
        json_data = json.load(json_file)
        clip_name = json_data["metaData"]['name']
        video.append(clip_name)
        # 첫번째의 start
        data_start = json_data["data"][0]['start']
        # 마지막의 end
        data_end = json_data["data"][-1]['end']
        clip_start = math.trunc(data_start * 10) / 10
        clip_end = math.ceil(data_end * 10) / 10
        start.append(clip_start)
        end.append(clip_end)

In [10]:
len(video), len(start), len(end)

(120, 120, 120)

#### 이미지에서 frame 추출 저장하기 -- 앞 뒤 정자세 너무 잘리는 느낌인데...

In [21]:
import cv2
from PIL import Image

In [33]:
path = root + DATA + '/영상/'
img_path = root + DATA + '/Image/'
for v, s, e in zip(video, start, end):
    vidcap = cv2.VideoCapture(path + v)
    count = -1
    save_n = 0
    while(vidcap.isOpened()):
        if not(os.path.isdir(img_path + '%s' % v)): # 이미지 담길 폴더 이름은 영상 이름
            os.makedirs(os.path.join(img_path + '%s' % v))
 
        count+=1
        success, image = vidcap.read()
        if success:
            if s*30 <= count and count <= e*30:
                save_n+=1
                img_file_name = v.split('.')[0]+'_'+str(count).zfill(12)+'_frame.png'
                resize_image = cv2.resize(image, dsize=(512, 512))
                cv2.imwrite(img_path+v+'/'+img_file_name, resize_image)
        else:
            print('%s %d개 이미지 증 %d개 이미지 생성' %(v, count, save_n))
            cv2.destroyAllWindows()
            vidcap.release()

NIA_SL_SEN0001_REAL06_F.mp4 169개 이미지 증 25개 이미지 생성
NIA_SL_SEN0001_REAL10_F.mp4 148개 이미지 증 34개 이미지 생성
NIA_SL_SEN0002_REAL06_F.mp4 175개 이미지 증 28개 이미지 생성
NIA_SL_SEN0002_REAL10_F.mp4 140개 이미지 증 34개 이미지 생성


KeyboardInterrupt: 

#### 이미지에 맞춰서 json도 삭제

In [50]:
path = root + DATA + '/키포인트Json/'
for v, s, e in zip(video, start, end):
    json_file = path+v
    for i, json_file_name in enumerate(sorted(os.listdir(json_file))):
        if i < s * 30 or e * 30 < i: # 종료 시간보다 전이거나 후면 삭제!
            os.remove(path+v+'/'+json_file_name)

## 2. Keypoint json 전처리 Txt로 저장

In [34]:
import glob
import numpy as np

In [51]:
# 경로 설정
kp_path = root + DATA + '/키포인트Json' # 기존 키포인트 Json
new_kp_dir = root + DATA + '/키포인트Txt' # 새로 저장할 키포인트 txt
len(os.listdir(kp_path))

120

#### 키포인트Json 폴더 아래 모든 json 파일 불러오기

In [36]:
json_files = glob.glob(os.path.join(kp_path,'**','*.json'), recursive=True)

In [38]:
len(json_files)

21517

#### keypoints 처리(연결, 0값제외, 값 채우기)

In [39]:
keypointss = []
ZERO = False
for i, json_file in enumerate(json_files):
    with open(json_file, "r") as keypoints_json:
        data = json.load(keypoints_json)
 
    # pose에서 생략 가능한 하반신 30-35, 39-44, 57-74
    pose = data['people'][0]['pose_keypoints_2d']
    pose = pose[:30]+pose[36:39]+pose[45:57] # pose 75개 -> 45개
    face = data['people'][0]['face_keypoints_2d']
    hand_left = data['people'][0]['hand_left_keypoints_2d']
    hand_right = data['people'][0]['hand_right_keypoints_2d']
    
    # 전체 keypoints 합치고 confidence 값 제외
    entire_keypoints = np.array(pose+face+hand_left+hand_right, dtype=np.float32).reshape(-1,3)[:,:2].flatten() # 254
    keypointss.append(entire_keypoints) # 일단 keypointss 리스트에 다 저장 - 0값 채우기 위해서
    
    if i>1 and ZERO: # 앞에 0인 값이 있으면 i-2, i의 평균값으로 i-1의 0 채워주기
        zero_tf = keypointss[i-1]==0
        keypointss[i-1][zero_tf] = (keypointss[i-2][zero_tf]+keypointss[i][zero_tf])/2 # 앞,뒤 평균값으로 채워주기
        ZERO = False
        
    if 0 in entire_keypoints: # 0이 있으면 ZERO!
        ZERO = True

#### 0~1 scaling
- 이미지 크기 2048 1152
- 여기서 1보다 큰 값은 이미지 밖에 찍히는 부분!

In [40]:
def normalization(array):
    scale_array = array.copy()
    scale_array[::2] /= 2048 # x좌표
    scale_array[1::2] /= 1152 # y좌표
    return scale_array

In [41]:
for json_file, keypoints in zip(json_files, keypointss):
    if 0 in keypoints:
        print(json_file, keypoints)
        break

#### 화자 구분 one-hot -- 화자 몇 명으로?

In [42]:
def check_one_hot(file_name):
    txt = file_name.split('_')[3]
    tmp = []
    for c in txt:
        if c.isdigit():
            tmp.append(c)
    return int(''.join(tmp))

#### 저장

In [53]:
os.makedirs(new_kp_dir, exist_ok=True) # 폴더 만들고
new_kp_dir = new_kp_dir + '/'
for json_file, keypoints in zip(json_files, keypointss):
    txt_file_name = os.path.basename(os.path.dirname(json_file))
    
    one_hot = [0 for i in range(22)]
    person = check_one_hot(txt_file_name)
    one_hot[person-1] = 1
    
    f = open(new_kp_dir+txt_file_name[:-4]+'.txt', 'a')
    # 저장
    data = "%s, " %os.path.basename(json_file)[:-5] + ', '.join(list(map(str,normalization(keypoints)))) + ', '.join(list(map(str,one_hot))) +'\n' # keypoints 값에 normalization 적용해서
    f.write(data)
    f.close()

KeyboardInterrupt: 